In [6]:
import numpy as np
import pandas as pd
import umap
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import seaborn as sns
import matplotlib.pyplot as plt
#from .autonotebook import tqdm as notebook_tqdm

data = pd.read_csv(r'C:\Users\jdorv\Coding Fun\Bank Innovation\wrds_bank_data_MERGED.csv')


In [7]:
print(data.shape[0]/48)
sparce = []
for col in data.columns:
    if data[col].isnull().sum()/len(data) > 0.9:
        sparce.append(col)

print("Sparce Columns: ", sparce)

6176.3125
Sparce Columns:  ['riad4011', 'riad4059', 'riad4097', 'riad4146', 'riad4172', 'riad4235', 'riad4239', 'riad4341', 'riad4531', 'riad4617', 'riad4645', 'riad4653', 'riad4658', 'riad4659', 'riad4663', 'riad4668', 'riad4669', 'riad4797', 'riad4815', 'riad4843', 'riad4844', 'riad4845', 'riad4846', 'rcfd1_rcfd0010', 'rcfd1_rcfd0022', 'rcfd1_rcfd0070', 'rcfd1_rcfd0071', 'rcfd1_rcfd0073', 'rcfd1_rcfd0074', 'rcfd1_rcfd0081', 'rcfd1_rcfd0082', 'rcfd1_rcfd0083', 'rcfd1_rcfd0085', 'rcfd1_rcfd0090', 'rcfd1_rcfd0426', 'rcfd1_rcfd1248', 'rcfd1_rcfd1249', 'rcfd1_rcfd1250', 'rcfd1_rcfd1251', 'rcfd1_rcfd1252', 'rcfd1_rcfd1253', 'rcfd1_rcfd1254', 'rcfd1_rcfd1350', 'rcfd1_rcfd1590', 'rcfd1_rcfd1754', 'rcfd1_rcfd1773', 'rcfd1_rcfd2011', 'rcfd1_rcfd2150', 'rcfd1_rcfd3000', 'rcfd1_rcfd3123', 'rcfd2_rcfd1403', 'rcfd2_rcfd1406', 'rcfd2_rcfd1407', 'rcfd2_rcfd1410', 'rcfd2_rcfd1420', 'rcfd2_rcfd1460', 'rcfd2_rcfd2122', 'rcfd2_rcfd2170', 'rcfd2_rcfd3210', 'rcon1_rcon1350', 'rcon2_rcon2011']


In [8]:
def handle_split_codes(df):
    """
    Handle RCFD/RCON split that occurred around 2011.
    Strategy: Use RCFD (consolidated) as primary, fill with RCON if needed
    """
    print(f"\n{'='*80}")
    print("HANDLING SPLIT TIME CODES (2011 Transition)")
    print(f"{'='*80}")
    
    # Define split code pairs (RCFD consolidated, RCON domestic)
    split_pairs = [
        ('rcfd2170', 'rcon2170'),  # Total Assets
        ('rcfd2122', 'rcon2122'),  # Total Loans
        ('rcfd3210', 'rcon3210'),  # Total Equity
        ('rcfd3123', 'rcon3123'),  # Allowance
    ]
    
    for rcfd_col, rcon_col in split_pairs:
        if rcfd_col in df.columns and rcon_col in df.columns:
            # Fill missing RCFD with RCON (for consistency)
            original_na = df[rcfd_col].isna().sum()
            df[rcfd_col] = df[rcfd_col].fillna(df[rcon_col])
            filled = original_na - df[rcfd_col].isna().sum()
            
            if filled > 0:
                print(f"✓ {rcfd_col}: Filled {filled:,} missing values with {rcon_col}")
            
            # Drop the RCON duplicate to avoid redundancy
            df = df.drop(columns=[rcon_col])
    
    return df


In [ ]:
def calculate_ratios(df):
    """Calculate ratios using ONLY universally reported fields"""
    print(f"\n{'='*80}")
    print("CALCULATING FINANCIAL RATIOS (UNIVERSAL FIELDS ONLY)")
    print(f"{'='*80}")
    
    ratios = df.copy()
    
    def safe_divide(numerator, denominator):
        result = numerator / denominator
        result = result.replace([np.inf, -np.inf], np.nan)
        return result
    
    # === CORE INNOVATION METRICS (3 ratios) ===
    ratios['tech_investment_ratio'] = safe_divide(
        df['riad4092'], df['rcfd2_rcfd2170']
    ) * 1000
    print("✓ Tech Investment Ratio")
    
    ratios['nib_deposit_ratio'] = safe_divide(
        df['rcon2_rcon6631'], df['rcon2_rcon2200']
    ) * 100
    print("✓ NIB Deposit Ratio (Digital Banking Proxy)")
    
    ratios['service_charge_intensity'] = safe_divide(
        df['riad4080'], df['rcon2_rcon2200']
    ) * 1000
    print("✓ Service Charge Intensity")
    
    # === EFFICIENCY METRICS (2 ratios) ===
    ratios['efficiency_ratio'] = safe_divide(
        df['riad4093'], df['riad4074'] + df['riad4079']
    ) * 100
    print("✓ Efficiency Ratio")
    
    ratios['nonint_income_pct'] = safe_divide(
        df['riad4079'], df['riad4074'] + df['riad4079']
    ) * 100
    print("✓ Noninterest Income %")
    
    # === BALANCE SHEET (3 ratios) ===
    ratios['loans_to_assets'] = safe_divide(
        df['rcfd2_rcfd2122'], df['rcfd2_rcfd2170']
    ) * 100
    print("✓ Loans-to-Assets")
    
    ratios['equity_to_assets'] = safe_divide(
        df['rcfd2_rcfd3210'], df['rcfd2_rcfd2170']
    ) * 100
    print("✓ Equity-to-Assets")
    
    ratios['deposits_to_assets'] = safe_divide(
        df['rcon2_rcon2200'], df['rcfd2_rcfd2170']
    ) * 100
    print("✓ Deposits-to-Assets")
    
    # === PROFITABILITY (2 ratios) ===
    ratios['roa'] = safe_divide(
        df['riad4340'], df['rcfd2_rcfd2170']
    ) * 100
    print("✓ ROA")
    
    ratios['roe'] = safe_divide(
        df['riad4340'], df['rcfd2_rcfd3210']
    ) * 100
    print("✓ ROE")
    
    # === DEPOSIT MIX (1 ratio) ===
    ratios['nontrans_deposits_pct'] = safe_divide(
        df['rcon2_rcon2215'], df['rcon2_rcon2200']
    ) * 100
    print("✓ Nontransaction Deposits %")
    
    print(f"\n✓ Created {11} ratios using universally reported fields")
    return ratios

In [ ]:
def prepare_clustering_features(df):
    """Select universal coverage features only"""
    print(f"\n{'='*80}")
    print("PREPARING CLUSTERING FEATURES - UNIVERSAL COVERAGE ONLY")
    print(f"{'='*80}")
    
    # RIAD columns - Universal coverage (<2% missing)
    riad_universal = [
        'riad4010', 'riad4012', 'riad4020', 'riad4073', 'riad4074', 'riad4079',
        'riad4080', 'riad4092', 'riad4093', 'riad4107', 'riad4115', 'riad4135',
        'riad4150', 'riad4180', 'riad4217', 'riad4230', 'riad4266', 'riad4267',
        'riad4300', 'riad4301', 'riad4302', 'riad4313', 'riad4340', 'riad4356',
        'riad4415', 'riad4435', 'riad4436', 'riad4460', 'riad4470', 'riad4498',
        'riad4499', 'riad4507', 'riad4508', 'riad4518', 'riad4605', 'riad4608',
        'riad4628', 'riad4635', 'riad4638', 'riad4644', 'riad4769'
    ]
    
    # RCON columns - Universal coverage (<2% missing)
    rcon_universal = [
        'rcon1_rcon1590', 'rcon1_rcon1754', 'rcon1_rcon1766', 'rcon1_rcon1773',
        'rcon1_rcon3000', 'rcon1_rcon3123', 'rcon2_rcon0081', 'rcon2_rcon1420',
        'rcon2_rcon1460', 'rcon2_rcon2122', 'rcon2_rcon2150', 'rcon2_rcon2170',
        'rcon2_rcon2200', 'rcon2_rcon2202', 'rcon2_rcon2215', 'rcon2_rcon3210',
        'rcon2_rcon6631'
    ]
    
    # Ratios - Successfully calculated (<2% missing)
    ratios_clean = [
        'nib_deposit_ratio',
        'service_charge_intensity',
        'efficiency_ratio',
        'nonint_income_pct',
        'nontrans_deposits_pct'
    ]
    
    # Combine all feature columns
    feature_cols = riad_universal + rcon_universal + ratios_clean
    
    # Check which features actually exist in dataframe
    existing_features = [col for col in feature_cols if col in df.columns]
    missing_features = [col for col in feature_cols if col not in df.columns]
    
    print(f"\n📊 Feature selection:")
    print(f"  Expected: {len(feature_cols)} features")
    print(f"  Found: {len(existing_features)} features")
    if missing_features:
        print(f"  Missing: {len(missing_features)} features")
        print(f"    (These weren't created - likely ratio calculation issues)")
    
    # Create feature matrix with only existing features
    X = df[existing_features].copy()
    
    print(f"\n🔍 Missing value summary:")
    total_missing = X.isna().sum().sum()
    total_cells = X.shape[0] * X.shape[1]
    missing_pct = (total_missing / total_cells) * 100
    print(f"  Total missing values: {total_missing:,} ({missing_pct:.2f}% of all cells)")
    
    # Show features with >0% missing
    missing_by_col = X.isna().sum()
    if (missing_by_col > 0).any():
        print(f"\n  Features with missing values:")
        for col in missing_by_col[missing_by_col > 0].index:
            pct = (missing_by_col[col] / len(X)) * 100
            print(f"    {col}: {missing_by_col[col]:,} ({pct:.1f}%)")
    
    # Drop rows with ANY missing values
    print(f"\n{'='*80}")
    print("REMOVING ROWS WITH MISSING VALUES")
    print(f"{'='*80}")
    
    before_rows = len(X)
    X_clean = X.dropna()
    after_rows = len(X_clean)
    
    print(f"  Before: {before_rows:,} rows")
    print(f"  After:  {after_rows:,} rows")
    print(f"  Retained: {after_rows/before_rows*100:.1f}%")
    
    print(f"\n{'='*80}")
    print(f"✓ FINAL DATASET READY FOR CLUSTERING")
    print(f"{'='*80}")
    print(f"  Observations: {len(X_clean):,}")
    print(f"  Features: {len(X_clean.columns)}")
    print(f"  Missing values: {X_clean.isna().sum().sum()}")
    print(f"  Ready for: Z-score standardization → UMAP → HDBSCAN")
    
    return X_clean, existing_features


HANDLING SPLIT TIME CODES (2011 Transition)

CALCULATING FINANCIAL RATIOS (UNIVERSAL FIELDS ONLY)
✓ Tech Investment Ratio
✓ NIB Deposit Ratio (Digital Banking Proxy)
✓ Service Charge Intensity
✓ Efficiency Ratio
✓ Noninterest Income %
✓ Loans-to-Assets
✓ Equity-to-Assets
✓ Deposits-to-Assets
✓ ROA
✓ ROE
✓ Nontransaction Deposits %

✓ Created 11 ratios using universally reported fields

PREPARING CLUSTERING FEATURES - UNIVERSAL COVERAGE ONLY

📊 Feature selection:
  Expected: 63 features
  Found: 63 features

🔍 Missing value summary:
  Total missing values: 52,576 (0.28% of all cells)

  Features with missing values:
    riad4266: 3,749 (1.3%)
    riad4267: 3,749 (1.3%)
    riad4498: 1 (0.0%)
    riad4499: 1 (0.0%)
    riad4608: 3,749 (1.3%)
    riad4638: 3,749 (1.3%)
    riad4769: 2 (0.0%)
    rcon1_rcon1754: 246 (0.1%)
    rcon1_rcon1766: 3,749 (1.3%)
    rcon1_rcon1773: 246 (0.1%)
    rcon1_rcon3000: 4,201 (1.4%)
    rcon1_rcon3123: 4,201 (1.4%)
    rcon2_rcon0081: 4,201 (1.4%)
    r